In [1]:
import add_packages
import config
from pprint import pprint
import os, typing

from my_langchain import (
  document_loaders, text_splitters, text_embedding_models, vector_stores, 
  chat_models, prompts, utils, output_parsers, agents, memories, chains,
  runnables, agent_tools
)

# LangChain

## Tool use (Usecase)

### Quickstart

Create Chains and Agents to call Tools, which can be APIs, functions, databases, etc. Tools extend a model's capabilities beyond text/messages. Prompt and parse-output a model correctly to choose the right tools and inputs.

#### Tool Creation

In [6]:
@agent_tools.tool
def tool_multiply(a: int, b: int) -> int:
  """Multiply two integers together."""
  return a*b

#### Chains

Create a chain for using a tool a fixed number of times, like a simple chain that multiplies user-specified numbers.


Using function calling APIs with LLM models that support it like OpenAI models.



In [17]:
# Define model and tools. 
model = chat_models.chat_openai

# Convert LangChain Tool to OpenAI JSONSchema function, bind as tools argument 
# for ChatOpenAI calls. Specify tool_choice for single Tool in initial chain to
# ensure consistent use. 
model_tools_equipped = model.bind_tools([tool_multiply], tool_choice="tool_multiply")

# Compose tool-calling model using JsonOutputToolsParser to convert OpenAI 
# function-calling response to list of {"type": "TOOL_NAME", "args": {...}} 
# dicts with tools and arguments.
chain = (
  model_tools_equipped 
  | output_parsers.JsonOutputToolsParser(key_name="tool_multiply", return_single=True)
  | tool_multiply
)

#### Agents

Chains are useful for specific tool sequences. In some cases, tool usage depends on input. Agents allow the model to determine tool usage frequency and order.

LangChain has built-in agents optimized for various use cases. 

OpenAI tools agent utilizes the new OpenAI tool-calling API. This differs from function-calling as the model can return multiple function invocations at once.

Agents can to use multiple tools. 

In [22]:
prompt = prompts.hub.pull("hwchase17/openai-tools-agent")

@agent_tools.tool
def tool_add(a: int, b: int) -> int:
  """Add two integers."""
  return a + b

@agent_tools.tool
def tool_exponentiate(base: int, exponent: int) -> int:
  """Exponentiate the base to the exponent power."""
  return base**exponent

tools = [
  tool_add,
  tool_exponentiate,
  tool_multiply,
]

model = chat_models.chat_openai
agent = agents.create_openai_tools_agent(model, tools, prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
queries = [
  ("Take 3 to the fifth power and multiply that by the sum of twelve and three, "
   "then square the whole result"),
]

for query in queries:
  agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `tool_exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `tool_add` with `{'a': 12, 'b': 3}`


15
Invoking: `tool_multiply` with `{'a': 243, 'b': 15}`


3645
Invoking: `tool_exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.


### Agents

Chains are useful for specific tool sequences. In some cases, tool usage depends on input. Agents allow the model to determine tool usage frequency and order.

LangChain has various built-in agents optimized for different use cases.

OpenAI tools agent utilizes OpenAI tool-calling API differs from function-calling as the model can return multiple function invocations at once.

In [24]:
#* TOOLS

@agent_tools.tool
def tool_add(a: int, b: int) -> int:
  """Add two integers."""
  return a + b

@agent_tools.tool
def tool_exponentiate(base: int, exponent: int) -> int:
  """Exponentiate the base to the exponent power."""
  return base**exponent

@agent_tools.tool
def tool_multiply(a: int, b: int) -> int:
  """Multiply two integers together."""
  return a*b

tools = [
  tool_add,
  tool_exponentiate,
  tool_multiply,
]


In [25]:
#* PROMPT
prompt = prompts.hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [26]:
#* AGENT
model = chat_models.chat_openai
agent = agents.create_openai_tools_agent(llm=model, tools=tools, prompt=prompt)
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
queries = [
  ("Take 3 to the fifth power and multiply that by the sum of twelve and three, "
   "then square the whole result"),
]

for query in queries:
  agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `tool_exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `tool_add` with `{'a': 12, 'b': 3}`


15
Invoking: `tool_multiply` with `{'a': 243, 'b': 15}`


3645
Invoking: `tool_exponentiate` with `{'base': 3645, 'exponent': 2}`


13286025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.


### Tool use without function calling


### Human-in-the-loop


### Parallel tool use
